In [1]:
pwd

'/home/wsuser/work'

In [2]:
import tensorflow as tf
tf.__version__

'2.7.2'

In [3]:
import keras
keras.__version__

'2.7.0'

In [4]:
import os
import numpy as np
import pandas as pd

In [5]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='NhA5XAu3PupIxjK1ipy7QqayZVZ4XXNlHfuJv-CTURwF',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'gesturerecognition-donotdelete-pr-bqiugumhf2mrwn'
object_key = 'Dataset.zip'

streaming_body_1 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [6]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_1.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [7]:
pwd

'/home/wsuser/work'

In [8]:
#Checks if the dataset got unzipped properly
filenames = os.listdir("/home/wsuser/work/Dataset/train")

In [9]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [10]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
x_train = train_datagen.flow_from_directory('/home/wsuser/work/Dataset/train/', target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')
x_test = test_datagen.flow_from_directory('/home/wsuser/work/Dataset/test/', target_size=(64, 64), batch_size=5, color_mode='grayscale', class_mode='categorical')

Found 594 images belonging to 6 classes.
Found 30 images belonging to 6 classes.


In [12]:
x_train.class_indices

{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5}

In [13]:
classifier = Sequential()

In [14]:
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 1) ,activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Flatten())

In [15]:
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=6, activation='softmax'))

In [16]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [17]:
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [18]:
classifier.fit_generator(
    generator=x_train, steps_per_epoch=len(x_train),
    epochs=20, validation_data=x_test, validation_steps=len(x_test)
)

/tmp/wsuser/ipykernel_4012/2617134232.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/20
119/119 [==============================] - 5s 42ms/step - loss: 1.4090 - accuracy: 0.4259 - val_loss: 0.6351 - val_accuracy: 0.8000
Epoch 2/20
119/119 [==============================] - 5s 38ms/step - loss: 0.7041 - accuracy: 0.7003 - val_loss: 0.5916 - val_accuracy: 0.8000
Epoch 3/20
119/119 [==============================] - 5s 41ms/step - loss: 0.5020 - accuracy: 0.7963 - val_loss: 0.6768 - val_accuracy: 0.7333
Epoch 4/20
119/119 [==============================] - 5s 41ms/step - loss: 0.3939 - accuracy: 0.8519 - val_loss: 0.4509 - val_accuracy: 0.8667
Epoch 5/20
119/119 [==============================] - 5s 40ms/step - loss: 0.3088 - accuracy: 0.8889 - val_loss: 0.3790 - val_accuracy: 0.8667
Epoch 6/20
119/119 [==============================] - 5s 40ms/step - loss: 0.2642 - accuracy: 0.8973 - val_loss: 0.4686 - val_accuracy: 0.8667
Epoch 7/20
119/119 [==============================] - 5s 40ms/step - loss: 0.1891 - accuracy: 0.9343 - val_loss: 0.3799 - val_accuracy: 0.9333

In [20]:
classifier.save('gesture.h5')

In [21]:
!tar -zcvf gesture-classifier.tgz gesture.h5

gesture.h5


In [22]:
ls

Dataset/  gesture-classifier.tgz  gesture.h5


In [23]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as f:
    f.write(model_json)

In [24]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 24.5 MB/s eta 0:00:01


In [25]:
from ibm_watson_machine_learning import APIClient

wml_credentials={
    
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"xhYfr2sw7BoEM8unaHSrXivomwlh4Cqyk0F7MzxcloSa"
}

In [26]:
client = APIClient(wml_credentials)

In [27]:
client

In [28]:
def guid_space_name(client,Gesture):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']['name']==Gesture)['metadata']['id'])

In [29]:
space_uid=guid_space_name(client,'gesture-recognition')
print("Space UID = " + space_uid)

Space UID = 9930b49f-907b-4a1f-b5f0-e146c7d081b1


In [30]:
client.set.default_space(space_uid)

'SUCCESS'

In [31]:
client.software_specifications.list(100)

-------------------------------  ------------------------------------  ----
NAME                             ASSET_ID                              TYPE
default_py3.6                    0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12        020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt       069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6          09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12       09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9        0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6            0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                       0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod     1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6                10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl        111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1

In [32]:
software_spec_uid = client.software_specifications.get_uid_by_name('tensorflow_rt22.1-py3.9')
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [34]:
model_details = client.repository.store_model(model='gesture-classifier.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:'tensorflow_rt22.1',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})

model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [35]:
model_id

'b551a542-6d4f-43d1-a1df-815fe17e7e8c'

In [36]:
client.repository.download(model_id, 'gesture-model.tar.gz')

Successfully saved model content to file: 'gesture-model.tar.gz'


'/home/wsuser/work/gesture-model.tar.gz'